# Работа с параллельным русско-цыганским корпусом в MongoDB 

In [3]:
!pip3 install pymorphy2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
!pip3 uninstall bson --y
!pip3 uninstall pymongo --y
!pip3 install pymongo

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 350 kB 1.8 MB/s            
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [5]:
from pprint import pprint

In [9]:
import requests
import re
import time
import datetime
from tqdm import tqdm
import pymorphy2

import pymongo
from bson import ObjectId

delcom=re.compile("<!--.+-->", re.S)

In [10]:
# Подключаемся к базе данных
client = pymongo.MongoClient('localhost', 27017)

In [702]:
db = client['test3_db']

In [703]:
rus = db.rus # тут будут лежать тексты на русском
roma = db.roma # тут будут лежать тексты на цыганском

### Подготовка данных

In [13]:
import pandas as pd

In [449]:
df = pd.read_csv('/Users/assmirnova/Downloads/corpus.csv', sep=',', encoding='utf-8')
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df.head()

,roma,rus,source_ru,source_roma,author,original_year,translation_year
0,Глава I. Бут бэрша одолэскэ палэ дрэ екх пэски...,﻿Глава I. Несколько лет тому назад в одном из ...,Дубровский,Дубровско,Александр Пушкин,1841,1935
1,"Лэскиро барвалыпэ, славутно родо и спхандыпэна...","Его богатство, знатный род и связи давали ему ...",Дубровский,Дубровско,Александр Пушкин,1841,1935
2,Пашылатунэ рая радымаса могискирдэ тэ кэрэн со...,Соседи рады были угождать малейшим его прихотя...,Дубровский,Дубровско,Александр Пушкин,1841,1935
3,Кирило Петровичё прилэлас подобострастиёскирэ ...,Кирила Петрович принимал знаки подобострастия ...,Дубровский,Дубровско,Александр Пушкин,1841,1935
4,"Никон на могискирдя тэ отпхэнэлпэ, коли Кирило...",Никто не дерзал отказываться от его приглашени...,Дубровский,Дубровско,Александр Пушкин,1841,1935


### Создадим класс для данных для их удобного преобразования в нужный формат

In [708]:
morph = pymorphy2.MorphAnalyzer()

class Rusroma:
    def __init__(self):
        self.title_rus = ""
        self.title_roma = ""
        self.author = ""
        self.year_original = ""
        self.year_translation = ""
        self.text_rus = ""
        self.text_roma = ""
    
def getDocumentText(row):
    docs = Rusroma()
    docs.title_rus = df.source_ru.loc[row]
    docs.title_roma = df.source_roma.loc[row]
    docs.author = df.author.loc[row]
    docs.year_original = df.original_year.loc[row]
    docs.year_translation = df.translation_year.loc[row]
    docs.text_rus = df.rus.loc[row]
    docs.text_roma = df.roma.loc[row]
    return docs


def getDocuments(df):
    for row in tqdm(range(len(df))):
        docs = getDocumentText(row)
        words = re.findall("([А-Яа-я]+(\-[А-Яа-я]+)?)", docs.text_rus)
        pos_tags = ''
        for w in words:
            wf = morph.parse(w[0])
            pos_tags += str(wf[0].tag.POS) + ' '
        
        
        res_rus={"title":docs.title_rus,
                 "author":docs.author,
                 "year":docs.year_original,
                 "sentence":docs.text_rus,
                 "pos_tags":pos_tags}
        text_id_rus = rus.insert_one(res_rus).inserted_id
        res_roma={"title":docs.title_roma,
                  "author":docs.author,
                  "year":docs.year_translation,
                  "sentence":docs.text_roma,
                  "original_text_id":text_id_rus}
        text_id_roma = roma.insert_one(res_roma).inserted_id

In [710]:
df['original_year'] = df['original_year'].astype(float)
df['translation_year'] = df['translation_year'].astype(float)

In [711]:
getDocuments(df)

100%|███████████████████████████████████████| 3640/3640 [00:48<00:00, 74.99it/s]


In [712]:
for t in rus.find()[:2]:
    pprint(t)

{'_id': ObjectId('6244bc2cf1b3d998f92bf58f'),
 'author': 'Александр Пушкин',
 'pos_tags': 'NOUN ADVB NOUN ADJF ADVB PREP ADJF PREP ADJF NOUN VERB ADJF ADJF '
             'NOUN NOUN NOUN NOUN ',
 'sentence': '\ufeffГлава I. Несколько лет тому назад в одном из своих '
             'поместий жил старинный русской барин, Кирила Петрович Троекуров.',
 'title': 'Дубровский',
 'year': 1841.0}
{'_id': ObjectId('6244bc2cf1b3d998f92bf591'),
 'author': 'Александр Пушкин',
 'pos_tags': 'NPRO NOUN ADJF VERB CONJ NOUN VERB NPRO ADJF NOUN PREP NOUN ADVB '
             'VERB NPRO NOUN ',
 'sentence': 'Его богатство, знатный род и связи давали ему большой вес в '
             'губерниях, где находилось его имение.',
 'title': 'Дубровский',
 'year': 1841.0}


In [713]:
for t in roma.find()[:2]:
    pprint(t)

{'_id': ObjectId('6244bc2cf1b3d998f92bf590'),
 'author': 'Александр Пушкин',
 'original_text_id': ObjectId('6244bc2cf1b3d998f92bf58f'),
 'sentence': 'Глава I. Бут бэрша одолэскэ палэ дрэ екх пэскирэ филатинэндыр '
             'джиндя пхурано русско рай, Кирило Петровичё Троекурово.',
 'title': 'Дубровско',
 'year': 1935.0}
{'_id': ObjectId('6244bc2cf1b3d998f92bf592'),
 'author': 'Александр Пушкин',
 'original_text_id': ObjectId('6244bc2cf1b3d998f92bf591'),
 'sentence': 'Лэскиро барвалыпэ, славутно родо и спхандыпэна дэнас лэскэ баро '
             'тырдыпэ дрэ губерния, кай сыс лэскири филатин.',
 'title': 'Дубровско',
 'year': 1935.0}


### Интерфейс с CRUD (Create, Read, Update, Delete)

#### Можем добавлять данные (в формате словаря)

In [664]:
def create(collection1, collection2, data1, data2):
    id_rus = collection1.insert_one(data1).inserted_id 
    data2['original_text_id'] = id_rus # запоминаем id русского текста и кладем к цыганским данным,
                                       # чтобы их связать между собой
    id_roma = collection2.insert_one(data2).inserted_id
    print('Created successfully.')

In [665]:
create(rus,roma,
       {"title": 'Питер Пэн', 
        "author":'Джеймс Мэтью Барри',
        "year":float(1902),
        "sentence":'Весь мир сделан из веры, доверия и пыльцы фей.'},
       {"title":'Peter Pan',
        "year":float(1999),
        "sentence":'All the world is made of faith, and trust, and pixie dust.'})

Created successfully.


#### Читать данные по найденным парам ключ-значение

In [666]:
def read(collection, key, value):
    if collection.count_documents({key:value}) != 0:
        return collection.find({key:value})[0]
    else:
        return 'No such document found.'

Выведем из двух коллекций оригинал и перевод добавленного предложения

In [667]:
rus_updated_res = read(rus, 'author', 'Джеймс Мэтью Барри')
print('Оригинал:', rus_updated_res['sentence'])

Оригинал: Весь мир сделан из веры, доверия и пыльцы фей.


In [668]:
id_updated_rus = rus_updated_res['_id']
print(id_updated_rus) # запомнили id, чтобы потом удалить этот документ
print('Перевод:', read(roma, 'original_text_id', id_updated_rus)['sentence'])

6244b2ecf1b3d998f92bf58d
Перевод: All the world is made of faith, and trust, and pixie dust.


In [669]:
read(rus, 'author', 'Петя Петров') # Такого нет, все выводится правильно

'No such document found.'

#### Изменение данных (отдельно для каждой коллекции, мало ли что захочется поменять)

In [670]:
def update(collection, _id, key, value):
    if collection.count_documents({'_id':ObjectId(_id)}) != 0:
        collection.update_one({"_id": ObjectId(_id)}, {"$set":{key:value}})
        return 'Updated successfully.'
    else:
        return 'No such id found.'

In [671]:
update(rus, '6244b2ecf1b3d998f92bf58d', 'author', 'Петя Петров')

'Updated successfully.'

In [672]:
read(rus, '_id', ObjectId('6244b2ecf1b3d998f92bf58d'))

{'_id': ObjectId('6244b2ecf1b3d998f92bf58d'),
 'title': 'Питер Пэн',
 'author': 'Петя Петров',
 'year': 1902.0,
 'sentence': 'Весь мир сделан из веры, доверия и пыльцы фей.'}

#### Удаление документов из двух коллекций по заданному условию ключ-значение (для русской коллекции)

In [673]:
def delete(collection1, collection2, key, value):
    if collection1.count_documents({key:value}) != 0:
        id_to_delete = read(collection1, key, value)['_id']
        collection1.delete_one({key:value})
        collection2.delete_one({'original_text_id':id_to_delete})
        print('Deleted successfully.')

In [674]:
delete(rus, roma, 'author', 'Петя Петров')

Deleted successfully.


In [675]:
read(rus, 'author', 'Петя Петров')

'No such document found.'

In [676]:
read(roma, 'original_text_id', '6244b2ecf1b3d998f92bf58d')

'No such document found.'

проверим на наличие документов (до update и после него) после операции удаления

In [677]:
read(rus, 'author', 'Петя Петров')

'No such document found.'

In [678]:
read(rus, 'author', 'Джеймс Мэтью Барри')

'No such document found.'

### Функции для интерфейса (приложения)

In [679]:
rus.create_index([('sentence', 'text')], default_language="russian")

'sentence_text'

In [680]:
roma.create_index([('sentence', 'text')])

'sentence_text'

#### Поиск по русским токенам (и даже словосочетаниям)

In [681]:
def rus_search(query):
    output = []
    for t in rus.find({"$text": {"$search": query}}):
        found = roma.find_one({'original_text_id': t['_id']})
        output.append({'rus': t['sentence'], 
                    'roma': found['sentence'], 
                    'author': t['author'],
                    'title_rus': t['title'],
                    'year_original': t['year'],
                    'title_roma': found['title'],
                    'year_translation': found['year']})
    return output

In [682]:
rus_search('жидкая грязь')

[{'rus': 'Опять она видит шоссе, покрытое жидкою грязью.',
  'roma': 'Отэнчя ёй дыкхэла шоссе, учякирдо жыдконэ блатаса.',
  'author': 'Антон Чехов',
  'title_rus': 'Спать хочется. Ванька',
  'year_original': 1888.0,
  'title_roma': 'Тэ совэм камэлпэ. Ванька',
  'year_translation': 1934.0},
 {'rus': 'Вдруг люди с котомками и тени надают на землю в жидкую грязь.',
  'roma': 'Екхатыр мануша гонэнца и тененца пэрна прэ пхув дрэ жыдко блата.',
  'author': 'Антон Чехов',
  'title_rus': 'Спать хочется. Ванька',
  'year_original': 1888.0,
  'title_roma': 'Тэ совэм камэлпэ. Ванька',
  'year_translation': 1934.0},
 {'rus': 'Но вот подул ветер, пропали облака, и Варька видит широкое шоссе, покрытое жидкою грязью; по шоссе тянутся обозы, плетутся люди с котомками на спинах, носятся взад и вперед какие-то тени; по обе стороны сквозь холодный, суровый туман видны леса.',
  'roma': 'Нэ окэ запхурдыя балвал, хасинэ облаки и Варька дыкхэла буґло шоссе учякирдо жыдконэ блатаса; пиро шоссе тырдэнапэ обо

#### Поиск по цыганским токенам (ищет только слово в слово)

In [683]:
from bson.objectid import ObjectId
def roma_search(query):
    output = []
    for t in roma.find({"$text": {"$search": query}}):
        found = rus.find_one({'_id':t['original_text_id']})
        output.append({'rus':found['sentence'], 
                    'roma': t['sentence'], 
                    'author': found['author'],
                    'title_rus':found['title'],
                    'year_original': found['year'],
                    'title_roma':t['title'],
                    'year_translation':t['year']})
    return output

In [684]:
roma_search('лондо мачё')

[{'rus': 'Выпей-ка огуречного рассолу с медом, а всего бы лучше опохмелиться полстаканчиком настойки.',
  'roma': 'Выпи-ка граставицэндыр лондо пайори ягвинэса, а фэдыр сарэстыр тэ выпьес паш мусэроро бравинта.',
  'author': 'Александр Пушкин',
  'title_rus': 'Капитанская дочка',
  'year_original': 1836.0,
  'title_roma': 'Капитаноскири чяй',
  'year_translation': 1936.0},
 {'rus': 'А на неделе хозяйка велела мне почистить селедку, а я начал с хвоста, а она взяла селедку и ейной мордой начала меня в харю тыкать.',
  'roma': 'А про курко хуланы пхэндя мангэ тэ ошулавав лондо мачё, а мэ лыёмпэ порьятыр, а ёй лыя мачё и лэскирэ шэрэса лыя ман дро муй тэ пэкавэл.',
  'author': 'Антон Чехов',
  'title_rus': 'Спать хочется. Ванька',
  'year_original': 1888.0,
  'title_roma': 'Тэ совэм камэлпэ. Ванька',
  'year_translation': 1934.0}]

#### Поиск по тегам, вывод предложений на русском и их перевода

In [739]:
def pos_search(pos_string):
    output = []
    for t in rus.find({'pos_tags': {'$regex': pos_string}}):
        found = roma.find_one({'original_text_id': t['_id']})
        output.append({'rus': t['sentence'], 
                    'roma': found['sentence'], 
                    'author': t['author'],
                    'title_rus': t['title'],
                    'year_original': t['year'],
                    'title_roma': found['title'],
                    'year_translation': found['year']})
    return output

In [741]:
for i in pos_search('NOUN NOUN')[:2]:
    pprint(i)

{'author': 'Александр Пушкин',
 'roma': 'Глава I. Бут бэрша одолэскэ палэ дрэ екх пэскирэ филатинэндыр джиндя '
         'пхурано русско рай, Кирило Петровичё Троекурово.',
 'rus': '\ufeffГлава I. Несколько лет тому назад в одном из своих поместий жил '
        'старинный русской барин, Кирила Петрович Троекуров.',
 'title_roma': 'Дубровско',
 'title_rus': 'Дубровский',
 'year_original': 1841.0,
 'year_translation': 1935.0}
{'author': 'Александр Пушкин',
 'roma': 'Пашылатунэ рая радымаса могискирдэ тэ кэрэн со наяви ваш лэскири '
         'угода; форитка чиновникэ трэнскирнаспэ дарипнастыр, коли кон '
         'рипирлас лэскиро лав;',
 'rus': 'Соседи рады были угождать малейшим его прихотям; губернские чиновники '
        'трепетали при его имени;',
 'title_roma': 'Дубровско',
 'title_rus': 'Дубровский',
 'year_original': 1841.0,
 'year_translation': 1935.0}


### Сортировка, группировка, агрегация данных

#### Группировка по авторам и сортировка по кол-ву произведений

In [685]:
res = rus.aggregate([
    {"$group" : {"_id":{"status":"$author"}, "count":{"$sum":1}}},
    {"$sort":{"count":-1}}
])
for i in res:
    print(i)

{'_id': {'status': 'Александр Пушкин'}, 'count': 2932}
{'_id': {'status': 'Лев Толстой'}, 'count': 304}
{'_id': {'status': 'Антон Чехов'}, 'count': 216}
{'_id': {'status': 'Александр Неверов'}, 'count': 188}


#### Группировка по авторам и подсчет средней длины предложений

In [688]:
author_len_res = rus.aggregate([
    {"$project": {
        "sentence": 1,
        "length": {"$strLenCP": "$sentence"},
        "author": 1}},
    {"$group": {"_id": "$author",
                "average_text_len":{"$avg":"$length"}}},
    {"$sort":{"average_text_len":-1}}
     ])
for i in author_len_res:
    print(i)

{'_id': 'Лев Толстой', 'average_text_len': 85.08881578947368}
{'_id': 'Александр Пушкин', 'average_text_len': 73.20770804911324}
{'_id': 'Антон Чехов', 'average_text_len': 72.39814814814815}
{'_id': 'Александр Неверов', 'average_text_len': 46.202127659574465}


In [689]:
author_len_res = roma.aggregate([
    {"$project": {
        "sentence": 1,
        "length": {"$strLenCP": "$sentence"},
        "author": 1}},
    {"$group": {"_id": "$author",
                "average_text_len":{"$avg":"$length"}}},
    {"$sort":{"average_text_len":-1}}
     ])
for i in author_len_res:
    print(i)

{'_id': 'Лев Толстой', 'average_text_len': 86.64802631578948}
{'_id': 'Антон Чехов', 'average_text_len': 75.75925925925925}
{'_id': 'Александр Пушкин', 'average_text_len': 75.2943383356071}
{'_id': 'Александр Неверов', 'average_text_len': 50.712765957446805}


#### Сортировка по годам написания и перевода

In [599]:
pipeline = [
    {
        '$lookup': {
            'from': 'roma', 
            'localField': '_id', 
            'foreignField': 'original_text_id', 
            'as': 'roma_joined'
        }
    }
]

results = list(rus.aggregate(pipeline))

sorted_by_year = sorted(results, key=lambda d: d['year'])
pprint(sorted_by_year[:2])

[{'_id': ObjectId('624492fff1b3d998f92bd7b6'),
  'author': 'Александр Пушкин',
  'roma_joined': [{'_id': ObjectId('624492fff1b3d998f92bd7b7'),
                   'original_text_id': ObjectId('624492fff1b3d998f92bd7b6'),
                   'sentence': 'Колежско регистраторо.',
                   'title': 'Станцыонно смотрителё',
                   'year': 1936.0}],
  'sentence': 'Коллежский регистратор.',
  'title': 'Станционный смотритель',
  'year': 1831.0},
 {'_id': ObjectId('624492fff1b3d998f92bd7b8'),
  'author': 'Александр Пушкин',
  'roma_joined': [{'_id': ObjectId('624492fff1b3d998f92bd7b9'),
                   'original_text_id': ObjectId('624492fff1b3d998f92bd7b8'),
                   'sentence': 'Кон, дрэ холы, на пучья лэндыр бибахталы '
                               'лылвари, собы тэ вчинэс дрэ адая лылвари ваш '
                               'начяльникэнгиро нартыма, ваш наковлыпэн?',
                   'title': 'Станцыонно смотрителё',
                   'year': 1936.0}

In [600]:
sorted_by_translation_year = sorted(results, key=lambda d: d['roma_joined'][0]['year'])
pprint(sorted_by_translation_year[:2])

[{'_id': ObjectId('624492faf1b3d998f92bd22e'),
  'author': 'Александр Неверов',
  'roma_joined': [{'_id': ObjectId('624492faf1b3d998f92bd22f'),
                   'original_text_id': ObjectId('624492faf1b3d998f92bd22e'),
                   'sentence': '\ufeffРеволюцыя.',
                   'title': 'Ракирибэна ваш тыкнэ чяворэнгэ',
                   'year': 1930.0}],
  'sentence': '\ufeffРеволюция.',
  'title': 'Неверов',
  'year': 1925.0},
 {'_id': ObjectId('624492faf1b3d998f92bd230'),
  'author': 'Александр Неверов',
  'roma_joined': [{'_id': ObjectId('624492faf1b3d998f92bd231'),
                   'original_text_id': ObjectId('624492faf1b3d998f92bd230'),
                   'sentence': 'Приавнэ, а одой мануша прэ гаса псирна, гиля '
                               'багана.',
                   'title': 'Ракирибэна ваш тыкнэ чяворэнгэ',
                   'year': 1930.0}],
  'sentence': 'Приехали, а там народ по улицам ходит, песий поют.',
  'title': 'Неверов',
  'year': 1925.0}]
